## Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import VarianceThreshold

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

## Load labels

In [2]:
y = pd.read_csv('your_path/label.csv')
y = y['class1'].to_numpy()
print(y.shape)
print(y)

(378,)
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 11  1 11  1 11  1  1  1  1  1  1  1  1 11  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1 11  1 11  1  1 11  1 11  1 11  1 11  1 11  1 11
  1 11  1  1 11  1 11  1  1 11  1  1 11  1 11  1 11  1 11  1  1 11  1 11
  1 11  1 11  1  1 11  1 11  1 11  1 11  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 11  1 11  1  1  1  1  1  1  1 11  1 11  1  1 11  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  

## Load data
Load gene.csv, methyl.csv, miRNA.csv for a cancer type, transpose the matrix and delete the first row. Row deletion because the first row contains the feature names.

In [3]:
# load .csv file gene
X_gene = pd.read_csv(r'your_path/gene.csv')
print(X_gene.shape)

# load .csv file methyl
X_methyl = pd.read_csv(r'your_path/methyl.csv')
print(X_methyl.shape)

# load .csv file miRNA
X_miRNA = pd.read_csv(r'your_path/miRNA.csv')
print(X_miRNA.shape)

(20531, 379)
(5000, 379)
(1046, 379)


In [4]:
X_gene_T = X_gene.transpose()
print(X_gene_T.shape)
print(X_gene_T)

X_methyl_T = X_methyl.transpose()
print(X_methyl_T.shape)
print(X_methyl_T)

X_miRNA_T = X_miRNA.transpose()
print(X_miRNA_T.shape)
print(X_miRNA_T)

(379, 20531)
                        0             1             2         3      \
Unnamed: 0       X..100130426  X..100133144  X..100134869  X..10357   
TCGA.2V.A95S.01           0.0        1.5051        3.7074   90.1124   
TCGA.2Y.A9GT.01           0.0           0.0        4.4833   95.5122   
TCGA.2Y.A9GU.01           0.0        5.7222        5.1216   61.6679   
TCGA.2Y.A9GV.01           0.0       11.4975         5.423   104.467   
...                       ...           ...           ...       ...   
TCGA.ZS.A9CD.01           0.0        4.2224        1.7881   64.5605   
TCGA.ZS.A9CE.01           0.0           0.0        8.7527   64.9891   
TCGA.ZS.A9CF.01           0.0        7.3398        9.8897  105.8856   
TCGA.ZS.A9CF.02           0.0        8.3127       11.4648  108.7824   
TCGA.ZS.A9CG.01           0.0           0.0        8.2262   57.1654   

                     4          5          6         7          8      \
Unnamed: 0        X..10431  X..136542  X..155060  X..26823  X

In [5]:
# GENE
# remove first row, containing names of features
X_gene_ = X_gene_T.iloc[1:, :]

# print shape and data to check if row containing names of features is removed
print('shape of X_gene: ', X_gene_.shape)
print('X_gene: \n', X_gene_.iloc[:10, :10])

# METHYL
# remove first row, containing names of features
X_methyl_ = X_methyl_T.iloc[1:, :]

# print shape and data to check if row containing names of features is removed
print('shape of X_methyl: ', X_methyl_.shape)
print('X_methyl: \n', X_methyl_.iloc[:10, :10])

# MIRNA
# remove first row, containing names of features
X_miRNA_ = X_miRNA_T.iloc[1:, :]

# print shape and data to check if row containing names of features is removed
print('shape of X_miRNA: ', X_miRNA_.shape)
print('X_miRNA: \n', X_miRNA_.iloc[:10, :10])

shape of X_gene:  (378, 20531)
X_gene: 
                       0        1       2         3          4    5         6  \
TCGA.2V.A95S.01     0.0   1.5051  3.7074   90.1124  1017.1038  0.0  141.3911   
TCGA.2Y.A9GT.01     0.0      0.0  4.4833   95.5122   742.4344  0.0    95.046   
TCGA.2Y.A9GU.01     0.0   5.7222  5.1216   61.6679  1186.9807  0.0  280.2709   
TCGA.2Y.A9GV.01     0.0  11.4975   5.423   104.467   878.1726  0.0  282.5719   
TCGA.2Y.A9GW.01     0.0   3.9234  7.5709   99.4866   690.4215  0.0   68.9655   
TCGA.2Y.A9GX.01     0.0      0.0  8.4848   75.0909   680.6061  0.0  124.8485   
TCGA.2Y.A9GY.01     0.0   3.5137  6.1275   45.1098  1199.2501  0.0  224.4242   
TCGA.2Y.A9GZ.01  0.7167   3.8225  9.1468   87.6928   546.7577  0.0  522.8669   
TCGA.2Y.A9H0.01     0.0   3.4345  8.4517  138.2358  1361.9018  0.0   44.4166   
TCGA.2Y.A9H1.01     0.0   0.7285  3.8971   35.8485  1221.1622  0.0   31.2229   

                      7        8    9  
TCGA.2V.A95S.01  0.6516      0.0  0.0 

# Scaling

In [6]:
def scale_data(data):
    
    # compute mean of features
    mean = np.mean(data, axis=1)
       
    # compute variance of features
    variance = np.var(data, axis=1)
    
    # compute standard deviation of features
    std = np.sqrt(variance)
    
    # scale data
    scaled = (data.T - mean) / std
   
    return scaled

In [7]:
X_gene_scaled = scale_data(X_gene_).T

X_methyl_scaled = scale_data(X_methyl_).T

X_miRNA_scaled = scale_data(X_miRNA_).T

# Feature selection VarianceThreshold

In [61]:
# set variance threshold
vt = VarianceThreshold(threshold=0.75)

# GENE
try:
    X_gene_varthr = vt.fit_transform(X_gene_scaled)
    print('Shape gene: ', X_gene_varthr.shape)
    np.savetxt("your_path/gene_FS.csv", X_gene_varthr.T, delimiter=", ")

except ValueError:
    print("No feature is strong enough to keep. Shape gene: ", X_gene_scaled.shape)
    np.savetxt("your_path/gene_FS.csv", X_gene_scaled.T, delimiter=", ")

# METHYL
try:
    X_methyl_varthr = vt.fit_transform(X_methyl_scaled)
    print('Shape methyl: ', X_methyl_varthr.shape)
    np.savetxt("your_path/methyl_FS.csv", X_methyl_varthr.T, delimiter=", ")

except ValueError:
    print("No feature is strong enough to keep. Shape methyl: ", X_methyl_scaled.shape)
    np.savetxt("your_path/methyl_FS.csv", X_methyl_scaled.T, delimiter=", ")

# MIRNA
try:
    X_miRNA_varthr = vt.fit_transform(X_miRNA_scaled)
    print('Shape miRNA: ', X_miRNA_varthr.shape)
    np.savetxt("your_path/miRNA_FS.csv", X_miRNA_varthr.T, delimiter=", ")

except ValueError:
    print("No feature is strong enough to keep. Shape miRNA: ", X_miRNA_scaled.shape)
    np.savetxt("your_path/miRNA_FS.csv", X_miRNA_scaled.T, delimiter=", ")

Shape gene:  (378, 310)
Shape methyl:  (378, 1503)
Shape miRNA:  (378, 18)


# Feature selection SelectKBest

In [73]:
# 0.3 0.5 0.4
# GENE
kbest = SelectKBest(mutual_info_classif, k=round(X_gene_scaled.shape[1]*0.3))
X_gene_kbest = kbest.fit_transform(X_gene_scaled, y)
print('Shape gene: ', X_gene_kbest.shape)
np.savetxt("your_path/gene_FS.csv", X_gene_kbest.T, delimiter=", ")

# METHYL
kbest = SelectKBest(mutual_info_classif, k=round(X_methyl_scaled.shape[1]*0.5))
X_methyl_kbest = kbest.fit_transform(X_methyl_scaled, y)
print('Shape methyl: ', X_methyl_kbest.shape)
np.savetxt("your_path/methyl_FS.csv", X_methyl_kbest.T, delimiter=", ")

# MIRNA
kbest = SelectKBest(mutual_info_classif, k=round(X_miRNA_scaled.shape[1]*0.4))
X_miRNA_kbest = kbest.fit_transform(X_miRNA_scaled, y)
print('Shape miRNA: ', X_miRNA_kbest.shape)
np.savetxt("your_path/miRNA_FS.csv", X_miRNA_kbest.T, delimiter=", ")

Shape gene:  (378, 6159)
Shape methyl:  (378, 2500)
Shape miRNA:  (378, 418)
